In [1]:
# Load required packages
library(dplyr,warn.conflicts=F)
library(stringr,warn.conflicts=F)
library(psych,warn.conflicts=F)
library(lplyr,warn.conflicts=F)

In [17]:
# load true results
data.truth <- readRDS('datafiles_rds/TourneyTrueResults.rds')

In [3]:
# load results
data.sagarin <- readRDS('predictions/Sagarin_bestqtl.rds')
data.dunkle <- readRDS('predictions/Dunkle_bestqtl.rds')

In [32]:
data.sagarin %>% 
    select(-qtl,PredSAG=Pred,ProbSAG=Prob) %>%
    inner_join(data.dunkle,by='ID') %>%
    select(-qtl,PredDUN=Pred,ProbDUN=Prob) -> df.mix

#### Equi Mix

In [33]:
df.mix %>% 
    mutate(ProbMIX=ifelse(abs(ProbSAG-0.5)>abs(ProbDUN-0.5),ProbSAG,ProbDUN)) %>% 
    select(-PredSAG,-PredDUN) %>%
    mutate(PredMIX=ifelse(ProbMIX>0.5,1,0)) -> df.mix
df.mix %>%
    inner_join(data.truth,by='ID') -> df.mix

In [35]:
# compute accuracy & Logloss per qtl to find the best one
df.mix %>% 
    mutate(OK=ifelse(PredMIX==Target,1,0)) %>%
    mutate(LogLoss=Target*log(ProbMIX) + (1-Target)*log(1-ProbMIX)) %>%
    summarise(TC=sum(OK),N=n(),FC=N-TC,Acc=TC/N,LogLoss=-mean(LogLoss)) %>%
    select(TC,FC,N,Acc,LogLoss) -> df.perf
df.perf %>% arrange(LogLoss) %>% head(5)

TC,FC,N,Acc,LogLoss
261,74,335,0.7791045,0.4672445


#### Champion vs Outsider